# Use Auto Arima (Autoregressive Integrated Moving Average) to Predict Revenue 

In this project, I created a Python notebook for predicting financial revenue of Bank using Auto Arima statistical method for time series forecasting as we take into account the past values to predict the future values of our bank revenues.

- p (past values used for forecasting the next value)

- q (past forecast errors used to predict the future values)

- d (order of differencing)


## Understanding the Problem Statement

Our focus is to do the technical analysis by analyzing the company’s future profitability on the basis of its current business financial performance by reading the charts and using statistical figures to identify the trends. 



## Data Sources:
* revenue_2009_2016.csv :  Processed data as an output from the data-preparation. Notebook available [here.](https://github.com/CFerraren/PyBank/blob/master/1-Data_Prep.ipynb)




### Task is to create a Python script that analyzes the records to calculate each of the following:

- Use auto ARIMA which automatically selects the best combination of (p,q,d) that provides the least error. 

- Split our data into train and validation sets to verify our predictions.


- Plot results.

### Changes:

- 02-12-2018: Started the project

- 12-11-2018: Updated the project using Python Pandas and added visualization using matplotlib, Tableau, and Univariate Linear Regression Machine learning to predict future bank revenue.


---

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import packages
import numpy as np
import pandas as pd
import os

In [3]:
#to plot within the notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#setting figure size
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10

In [5]:
#use seaborn to prettify the plots
import seaborn as sns
sns.set_style('whitegrid')

In [6]:
#display pd typeformat
pd.set_option('display.float_format', '{:,.0f}'.format)

In [7]:
#directory and filename
dir = 'data/processed/'
file = 'revenue_2009_2016.csv'

In [8]:
#load csv into dataframe and print the head
#parse the date colum to datetime format
#set 'Date' column to index
df = pd.read_csv(os.path.join(dir, file), parse_dates=['Date'])

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Revenue,86,"650,527","680,460","-1,063,151","279,551","686,900","1,118,162","2,211,086"


In [10]:
#make a copy
data = df.copy()
data.set_index('Date', inplace=True)

---

## Time Series Analysis

### Trend Analysis
-depicts the variation in the output as time increases

In [ ]:
rvn = data[['Revenue']]

In [ ]:
rvn.rolling(6).mean().plot(linewidth=1, fontsize=12, color='#2ecc71')
plt.show()

### Seasonability Analysis

In [ ]:
#Using 1st discrete difference of object

rvn.diff(periods=4).plot(linewidth=1, fontsize=12,  color='#2ecc71')
plt.show()

In [ ]:
#Autocorrelation

pd.plotting.autocorrelation_plot(rvn)
plt.show()

***Autocorrelation plot** is another way for checking randomness in time series. This is done by computing autocorrelations for data values at varying time lags. If time series is random, such autocorrelations should be near zero for any and all time-lag separations. If time series is non-random then one or more of the autocorrelations will be significantly non-zero.

In [ ]:
pd.plotting.lag_plot(rvn)
plt.show()

---
**Lag plots** are used to check if a data set or time series is random. Random data should not exhibit any structure in the lag plot. 

In [ ]:
#Import statsmodel
import statsmodels.api as sm

- **Time Series decomposition** - allows us to  decompose our time series into three distinct components: trend, seasonality, and noise.

In [ ]:
decomposition = sm.tsa.seasonal_decompose(data['Revenue'], model='additive')
fig = decomposition.plot()
plt.show()

In [ ]:
#split into train and validation
train = data[:72]
valid = data[72:]

In [ ]:
#assign the feature and target
training = train['Revenue']
validation = valid['Revenue']

In [ ]:
#importing libraries
from pyramid.arima import auto_arima

In [ ]:
#create model
model = auto_arima(training, start_p=1, start_q=1,max_p=3, 
                   max_q=3, m=12,start_P=0, seasonal=True,d=1, 
                   D=1, trace=True,error_action='ignore',
                   suppress_warnings=True)

model.fit(training)

In [ ]:
#forecast
forecast = model.predict(n_periods=valid.shape[0])

In [ ]:
forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])

### Results

In [ ]:
rms=np.sqrt(np.mean(np.power((np.array(valid['Revenue'])-np.array(forecast['Prediction'])),2)))
rms

In [ ]:
#plot
plt.plot(train['Revenue'])
plt.plot(valid['Revenue'])
plt.plot(forecast['Prediction'])